In [1]:
# Import the required libraries

import os
import zipfile
import random
import tensorflow as tf
from shutil import copyfile, rmtree, move
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from functools import partial
from matplotlib.image import imread
import matplotlib.image as mpimg
from tensorflow import keras
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPool2D

Using TensorFlow backend.


In [17]:
source_path = '/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/source'
source_benign_path = os.path.join(source_path, 'benign')
source_melignant_path = os.path.join(source_path, 'melignant')
data_path = '/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/data'
train_path = os.path.join(data_path, 'train')
train_benign_path = os.path.join(train_path, 'benign')
train_melignant_path = os.path.join(train_path, 'melignant')
validation_path = os.path.join(data_path, 'validation')
validation_benign_path = os.path.join(validation_path, 'benign')
validation_melignant_path = os.path.join(validation_path, 'melignant')

# Create new directories for small train and validation data
s_data_path = '/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/s_data'
s_train_path = os.path.join(s_data_path, 's_train')
s_train_benign_path = os.path.join(s_train_path, 's_benign')
s_train_melignant_path = os.path.join(s_train_path, 's_melignant')
s_validation_path = os.path.join(s_data_path, 's_validation')
s_validation_benign_path = os.path.join(s_validation_path, 's_benign')
s_validation_melignant_path = os.path.join(s_validation_path, 's_melignant')

try:
# Create the directories
    os.mkdir(s_data_path)
    os.mkdir(s_train_path)
    os.mkdir(s_train_benign_path)
    os.mkdir(s_train_melignant_path)
    os.mkdir(s_validation_path)
    os.mkdir(s_validation_benign_path)
    os.mkdir(s_validation_melignant_path)
   
except OSError:
    pass

In [18]:
len_train_benign = len(os.listdir(train_benign_path))
len_train_melignant = len(os.listdir(train_melignant_path))
len_train_total = len_train_benign + len_train_melignant
train_melignant_benign_ratio =  len_train_melignant/len_train_benign

len_validation_benign = len(os.listdir(validation_benign_path))
len_validation_melignant = len(os.listdir(validation_melignant_path))
len_validation_total = len_validation_benign + len_validation_melignant
validation_melignant_benign_ratio =  len_validation_melignant/len_validation_benign

In [19]:
print('Total Train Benign:', len_train_benign)
print('Total Train Melignant:', len_train_melignant)
print('Train Total:', len_train_total)
print('Train Melignant/Benign:', train_melignant_benign_ratio)

print('\nTotal Validation Benign:', len_validation_benign)
print('Total Validation Melignant:', len_validation_melignant)
print('Validation Total:', len_validation_total)
print('Validation Melignant/Benign:',validation_melignant_benign_ratio)

Total Train Benign: 26033
Total Train Melignant: 467
Train Total: 26500
Train Melignant/Benign: 0.01793877002266354

Total Validation Benign: 6509
Total Validation Melignant: 117
Validation Total: 6626
Validation Melignant/Benign: 0.01797511138423721


In [28]:
def split_data(source, s_train, s_validation, split_size):

    list_source = os.listdir(source)
    randomized_list = random.sample(list_source, int(len(list_source)*0.1))
    
    for i, item in enumerate(randomized_list):
        if i < int(split_size*len(randomized_list)):
            copyfile(os.path.join(source,item), os.path.join(s_train,item))
        else:
            copyfile(os.path.join(source,item), os.path.join(s_validation,item))

split_size = .8
split_data(source_benign_path, s_train_benign_path, s_validation_benign_path, split_size)
split_data(source_melignant_path, s_train_melignant_path, s_validation_melignant_path, split_size)

In [29]:
len_s_train_benign = len(os.listdir(s_train_benign_path))
len_s_train_melignant = len(os.listdir(s_train_melignant_path))
len_s_train_total = len_train_benign + len_train_melignant
train_s_melignant_benign_ratio =  len_s_train_melignant/len_s_train_benign

len_s_validation_benign = len(os.listdir(s_validation_benign_path))
len_s_validation_melignant = len(os.listdir(s_validation_melignant_path))
len_s_validation_total = len_validation_benign + len_validation_melignant
validation_s_melignant_benign_ratio =  len_s_validation_melignant/len_s_validation_benign

In [30]:
print('Total s_Train Benign:', len_s_train_benign)
print('Total s_Train Melignant:', len_s_train_melignant)
print('Train s_Total:', len_s_train_total)
print('Train s_Melignant/Benign:', train_s_melignant_benign_ratio)

print('\nTotal s_Validation Benign:', len_s_validation_benign)
print('Total s_Validation Melignant:', len_s_validation_melignant)
print('Validation s_Total:', len_s_validation_total)
print('Validation s_Melignant/Benign:',validation_s_melignant_benign_ratio)

Total s_Train Benign: 2603
Total s_Train Melignant: 46
Train s_Total: 26500
Train s_Melignant/Benign: 0.017671917018824434

Total s_Validation Benign: 651
Total s_Validation Melignant: 12
Validation s_Total: 6626
Validation s_Melignant/Benign: 0.018433179723502304
